In [1]:
from skimage.metrics import structural_similarity as ssim
import imutils
import cv2
from PIL import Image
import requests
import os
import numpy as np

In [2]:
os.makedirs("pan_card_tampering/images", exist_ok=True)

In [3]:
original = Image.open(requests.get(
    'https://www.thestatesman.com/wp-content/uploads/2019/07/pan-card.jpg',
    stream=True).raw)
original = original.resize((500, 320))
original.save('pan_card_tampering/images/original.png')

In [4]:
tampered = Image.open(requests.get(
    'https://assets1.cleartax-cdn.com/s/img/20170526124335/Pan4.png',
    stream=True).raw)
tampered = tampered.resize((500, 320))
tampered.save('pan_card_tampering/images/tampered.png')

In [5]:
original_cv = cv2.imread('pan_card_tampering/images/original.png')
tampered_cv = cv2.imread('pan_card_tampering/images/tampered.png')

In [6]:
print("Original image pixel sum:", np.sum(original_cv))
print("Tampered image pixel sum:", np.sum(tampered_cv))

Original image pixel sum: 89757768
Tampered image pixel sum: 90504219


In [7]:
original_gray = cv2.cvtColor(original_cv, cv2.COLOR_BGR2GRAY)
tampered_gray = cv2.cvtColor(tampered_cv, cv2.COLOR_BGR2GRAY)

In [8]:
(score, diff) = ssim(original_gray, tampered_gray, full=True)
diff = (diff * 255).astype("uint8")

print("SSIM Score:", score)

SSIM Score: 0.3234859105524235


In [9]:
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# Find contours of differences
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)


In [ ]:
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(original_cv, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.rectangle(tampered_cv, (x, y), (x + w, y + h), (0, 0, 255), 2)

# Display the images
cv2.imshow("Original with Differences", original_cv)
cv2.imshow("Tampered with Differences", tampered_cv)
cv2.imshow("Difference Map", diff)
cv2.imshow("Threshold", thresh)

cv2.waitKey(0)
cv2.destroyAllWindows()